In [2]:
import os 
import cv2

In [3]:
from keras.layers import Dense, Flatten, MaxPooling2D, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib
from keras.models import model_from_json

## Loading trained model

In [6]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
labels_dict = {0 : 'anger', 1 : 'contempt', 2 : 'disgust', 3 : 'fear', 4 : 'happy',5 :'sadness',6 :'surprise'}
Gender_dict = {0:'male',1:'female'}

In [7]:
scaler = joblib.load('scaler.pkl')

In [8]:
json_file = open('emotion_model.json', 'r')
emotion_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(emotion_model_json)
# load weights into new model
emotion_model.load_weights("emotion_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [9]:
json_file = open('gender_model.json', 'r')
Gender_json = json_file.read()
json_file.close()
Gender_model = model_from_json(Gender_json)
# load weights into new model
Gender_model.load_weights("gender_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [10]:
json_file = open('age_model.json', 'r')
Age_json = json_file.read()
json_file.close()
Age_model = model_from_json(Age_json)
# load weights into new model
Age_model.load_weights("age_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [11]:
def detect(im):
    faces = face_clsfr.detectMultiScale(im)

    for (x, y, w, h) in faces:
        cv2.rectangle(im,(x,y),(x+w,y+h),(0, 0, 225),2)
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(224,224))
        reshaped=np.expand_dims(resized, axis = 0)
        result=emotion_model.predict(reshaped)
        label=np.argmax(result)

        age = str(int(np.round(scaler.inverse_transform(Age_model.predict(reshaped))[0][0])))

        gender_prob = Gender_model.predict(reshaped)[0][0]
        if gender_prob < 0.5:
            gender = 0
        else:
            gender = 1

        gender = Gender_dict[gender]

        
        cv2.putText(im,labels_dict[label] , (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        cv2.putText(im,"Gender: "+gender , (x, y-30),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        cv2.putText(im,"Age: " + age , (x, y-50),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    return im

# Detection from local video

In [13]:
import imageio
# Read Video
reader = imageio.get_reader('WIN_20210331_21_25_54_Pro.mp4')
fps = reader.get_meta_data()['fps']
writer = imageio.get_writer('output.mp4', fps = fps)
for i, frame in enumerate(reader):
    if i%10 == 0:
      print(i)
    frame = detect(frame)
    writer.append_data(frame)
writer.close()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160


# Detection from realtime

In [ ]:

video_capture = cv2.VideoCapture(0) 

while True: 
    _, frame = video_capture.read()
    frame=cv2.flip(frame,1,1)
    frame = detect(frame)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 

video_capture.release()
cv2.destroyAllWindows() 